In [163]:
import time
from urllib.request import Request, urlopen
from urllib.error import HTTPError
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
options = Options()
options.binary_location = 'geckodriver'
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm

In [164]:
pages = 100

data_1 = []
done = False
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}
base_url = "https://www.argenprop.com/inmuebles/venta/buenos-aires"
for i in range(1, pages + 1):

    response = requests.get(f"{base_url}?pagina-{i}", headers=headers)
    soup = bs(response.text, 'html5lib')
    listings = soup.find_all('div', class_=lambda x:x=='listing__item' if x else False)
    data_1.append(listings)
    if done:
        if len(data_1) == 0:
            para_imprimir_al_final = 'No se encontraron publicaciones nuevas.'
        else:
            para_imprimir_al_final = f'Se encontraron {len(data_1)} publicaciones nuevas.'
        break
data = []
for listings in data_1:
    for listing in tqdm(listings):
        try:
            link_suffix = 'argenprop.com' + listing.find('a').get('href')
            link = 'https://www.argenprop.com' + link_suffix
            response2 = requests.get(f"Https://www.{link_suffix}",headers=headers)
            soup = bs(response2.text, 'html5lib')
            titulo = soup.find('h2', class_='titlebar__title').get_text(strip=True)
            precio = soup.find('div', class_='titlebar__price-mobile').get_text(strip = True)
            direccion = soup.find('h2',class_ = 'titlebar__address').get_text(strip = True)
            property_data = {
                'Titulo':titulo,
                'Precio':precio,
                'Link':link,
                'Dirección':direccion
            }
            features = soup.find('ul', class_='property-main-features')
            for feature in features.find_all('li'):
                titulo_caracteristica = feature.get('title')
                if titulo_caracteristica:
                    valor = feature.find('div', class_='desktop').find('p', class_='strong').get_text(strip=True)
                    property_data[titulo_caracteristica] = valor
            data.append(property_data)
        except:
            
            print(listing.find('a').get('href'))
            pass
df = pd.DataFrame(data)
df.to_excel(r'BD_Finales/argenprop.xlsx')